In [ ]:
from src.downloader import *
import numpy as np
import easyocr
from src.recognizer import Recognizer

In [ ]:
fpath = download('https://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20210904112822A5.pdf')
images = convert_image(fpath)
images.shape

In [ ]:
reader = easyocr.Reader(
  ['th'],
  recog_network='thai-vl',
  model_storage_directory='src/model',
  user_network_directory='src/model',
  detect_network='dbnet18'
  )

In [ ]:
recognizer = Recognizer(reader=reader, images=images)

In [ ]:
recognizer.recognize()